In [1]:
import numpy as np
import scipy

import sympy as sp

# 1) Правило Армихо

$x^0=(-1,1)$

$f:R^2\rightarrow R, f(x)=2x_1^2+x_1x_2+3x^2_2$

In [17]:
def func(x):
    return 2 * x[0] ** 2 + x[0] * x[1] + 3 * x[1] ** 2

def func_grad(x):
    return np.array([4*x[0] + x[1], x[0] + 6*x[1]])


In [18]:
%%time

x0 = np.array([-1.0, 1.0])

alpha = 1.
eps = 1e-4
theta = 0.5

x=x0
count = 0
N=1000


while count < N:
    print(f"i={count}, solution x1={x[0]} x2={x[1]}")
    grad=func_grad(x)
    new_x = x - alpha * func_grad(x)
    
    if func(new_x) > func(x) + eps * alpha*np.cross(-grad, grad ):
        alpha = theta * alpha
    
    count += 1
    if np.linalg.norm(grad) < eps:
        break
    else:
        x=new_x
print(f"solution x1={x[0]} x2={x[1]}")

i=0, solution x1=-1.0 x2=1.0
i=1, solution x1=2.0 x2=-4.0
i=2, solution x1=0.0 x2=7.0
i=3, solution x1=-1.75 x2=-3.5
i=4, solution x1=0.875 x2=2.1875
i=5, solution x1=-0.546875 x2=-1.3125
i=6, solution x1=0.328125 x2=0.79296875
i=7, solution x1=-0.1982421875 x2=-0.478515625
i=8, solution x1=0.11962890625 x2=0.288818359375
i=9, solution x1=-0.07220458984375 x2=-0.17431640625
i=10, solution x1=0.0435791015625 x2=0.1052093505859375
i=11, solution x1=-0.026302337646484375 x2=-0.06349945068359375
i=12, solution x1=0.015874862670898438 x2=0.03832530975341797
i=13, solution x1=-0.009581327438354492 x2=-0.023131370544433594
i=14, solution x1=0.0057828426361083984 x2=0.01396101713180542
i=15, solution x1=-0.003490254282951355 x2=-0.00842621922492981
i=16, solution x1=0.0021065548062324524 x2=0.0050856731832027435
i=17, solution x1=-0.0012714182958006859 x2=-0.003069475293159485
i=18, solution x1=0.0007673688232898712 x2=0.001852592220529914
i=19, solution x1=-0.0004631480551324785 x2=-0.0011181

# 2) Метод Ньютона

$x^0=(1,1)$

$f:R^2\rightarrow R, f(x)=x_1^2+e^{x^2_2}$

In [19]:
def func(x):
    return np.array(x[0] ** 2 + np.exp(x[1] ** 2))

def jacob(x):
    return np.array([2 * x[0],2 * x[1] *np.exp(x[1]**2)])

def hessian(x):
    return np.array([2, 2 * np.exp(x[1] ** 2) * (1 + 2 * x[1] ** 2)])


In [34]:
%%time
x0 = np.array([1.0, 1.0])
count = 0
precision=1e-10
N=1000

x=x0

while count<N:
    print(f"i={count}, solution x1={x[0]} x2={x[1]}")
    
    x=x-jacob(x)/hessian(x)
    if np.linalg.norm(jacob(x)) < precision:
        break

    count += 1


print(f"Solution x1={x[0]} x2={x[1]}")

i=0, solution x1=1.0 x2=1.0
i=1, solution x1=0.0 x2=0.6666666666666667
i=2, solution x1=0.0 x2=0.3137254901960785
i=3, solution x1=0.0 x2=0.0515989241825866
i=4, solution x1=0.0 x2=0.00027330369152017003
Solution x1=0.0 x2=4.082878197857187e-11
CPU times: user 200 µs, sys: 21 µs, total: 221 µs
Wall time: 205 µs


# 3) Метод сопряженных градиентов

$x^0=(1,1)$

$f:R^2\rightarrow R, f(x)=x_1^2+2x_2^2$

In [2]:
def func(x):
    return np.array(x[0] ** 2 + 2*x[1]**2)

def func_grad(x):
    return np.array([2*x[0], 4*x[1]])


In [26]:
%%time

x0 = np.array([1., 1.])

A = np.array([[1., 0.], [0., 2.]])
grad = -func_grad(x0)
beta = 0.
N=1000

count=0
x = x0

while count < N:
    if count >= 1:
        # параметр сопряженности
        beta = np.dot(A @ grad, func_grad(x)) / np.dot(A @ grad, grad)
        # beta=np.linalg.norm(func_grad(x))/np.linalg.norm(grad)
        
    grad = beta * grad - func_grad(x)
    alpha = - np.dot(A @ x, grad) / (np.dot(A @ grad, grad))
    x = x+  alpha * grad

    count += 1
    print(f"i={count}, solution x1={x[0]} x2={x[1]}")
    if np.linalg.norm(func_grad(x)) < 1e-9:
        break
    

print(f"Solution x1={x[0]} x2={x[1]}")

i=1, solution x1=0.4444444444444444 x2=-0.11111111111111116
i=2, solution x1=5.551115123125783e-17 x2=-6.938893903907228e-17
Solution x1=5.551115123125783e-17 x2=-6.938893903907228e-17
CPU times: user 800 µs, sys: 345 µs, total: 1.15 ms
Wall time: 1.03 ms


# 4) Метод условного градиента

$x^0=(0,0)$

$f:R^2\rightarrow R, f(x)=x_1^2-4x_1+x_2^2-2x_2$

In [37]:
def func(x):
    return np.array(x[0]**2 - 4*x[0] + x[1]**2 - 2*x[1])

def func_grad(x):
    return np.array([2*x[0] - 4, 2*x[1] - 2])


In [54]:
%%time

N=1000


x0 = np.array([0., 0.])
x = x0
count = 0
while count < N:

    d = scipy.optimize.minimize(lambda x_: np.dot(func_grad(x), x_ - x), x, bounds=((0, 1), (0, 2))).x - x

    lmbd = scipy.optimize.minimize(lambda alpha: func(x + alpha*d), 1.).x

    x_old=x
    x = x + lmbd * d

    print(f"i={count}, solution x1={x[0]} x2={x[1]}")

    count+=1
    if np.linalg.norm(x - x_old) <= 0.1:
        break

print(f'\nNumerical solution: x1 = {x[0]}, x2 = {x[1]}')


i=0, solution x1=0.8000000001441765 x2=1.600000000288353
i=1, solution x1=0.8923076945429015 x2=0.8615384444331233
i=2, solution x1=0.9151131228917716 x2=1.1026244098902052
i=3, solution x1=0.9293593412676299 x2=0.9175754522071443
i=4, solution x1=0.9392562675381645 x2=1.0692257359836421
i=5, solution x1=0.9465889144434657 x2=0.9401547278931397
i=6, solution x1=0.9522657295784155 x2=1.0528007379622073
i=7, solution x1=0.9568042683292424 x2=0.9527012307540692
i=8, solution x1=0.9605233974437478 x2=1.0428730878712649

Numerical solution: x1 = 0.9605233974437478, x2 = 1.0428730878712649
CPU times: user 15.7 ms, sys: 4.85 ms, total: 20.5 ms
Wall time: 17 ms


# 5) Метод квадратичного штрафа

$f(x)=2x^2_1+(x_2-1)^2\rightarrow min, x \in D=\{x \in R^2 | 2x_1+x_2=2\} $

In [6]:
def func(x):
    return 2 * x[0] ** 2 + (x[1] - 1) ** 2

def F(x):
    return 2 * x[0] + x[1]

def penalty(x):
    return 0.5 * np.linalg.norm(F(x)) ** 2

def quadratic(x,N=10000): 
    c_k = 1
    count = 0 
    iters=0 
    flag=True 
    eps=1e-5 
    while flag:
        print(f"Solution x1={x[0]} x2={x[1]}")
        curr_func = lambda x: func(x) + c_k * penalty(x) 
        x_old=x.copy()
        data = scipy.optimize.minimize(curr_func, x)
        iters+=data.nit
        x=data.x 
        c_k += 1e5
        count += 1
        if np.linalg.norm(x-x_old) < eps or count>N:
            flag=False
    print("Inner Iters num: " + str(iters))
    return x
x0 = np.array([-1.0, 1.0])
x = quadratic(x0)
print(f"Solution x1={x[0]:2f} x2={x[1]:2f}")

Solution x1=-1.0 x2=1.0
Solution x1=-0.1999996405150699 x2=0.7999997546943637
Solution x1=-0.3333399450214094 x2=0.6666865482538418
Inner Iters num: 11
Solution x1=-0.333341 x2=0.666686


# 6) Симплекс метод

In [56]:
x0_ = np.array([1., 0., 1., 0.])
c_ = np.array([-1., 3., 5., 1.])

A_eq_ = np.array([[1., 4., 4., 1.],
                  [1., 7., 8., 2.]])

b_eq_ = np.array([5., 9.])

In [57]:
res = scipy.optimize.linprog(c=c_, A_eq=A_eq_, b_eq=b_eq_, x0=x0_, method='highs')
res

/Users/dkagramanan/opt/anaconda3/envs/torch/lib/python3.9/site-packages/scipy/optimize/_linprog.py:582: OptimizeWarning: x0 is used only when method is 'revised simplex'. 
  warn(warning_message, OptimizeWarning)


           con: array([0., 0.])
 crossover_nit: 0
         eqlin:  marginals: array([-3.,  2.])
  residual: array([0., 0.])
           fun: 3.0
       ineqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
         lower:  marginals: array([0., 1., 1., 0.])
  residual: array([1., 0., 0., 4.])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 2
         slack: array([], dtype=float64)
        status: 0
       success: True
         upper:  marginals: array([0., 0., 0., 0.])
  residual: array([inf, inf, inf, inf])
             x: array([1., 0., 0., 4.])